# Project 3 - Part 4: Hypothesis Testing

# Business Problem

For this project, you have been hired to produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. Ultimately, you will use this database to analyze what makes a movie successful, and will provide recommendations to the stakeholder on how to make a successful movie.

## Part 4 

 For part 4 of the project, you will be using your MySQL database from part 3 to answer meaningful questions for your stakeholder. They want you to use your hypothesis testing and statistics knowledge to answer 3 questions about what makes a successful movie.

#### Questions to Answer

#### The stakeholder's first question is: does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates?

 - They want you to perform a statistical test to get a mathematically-supported answer.
 - They want you to report if you found a significant difference between ratings.
  - If so, what was the p-value of your analysis?
  - And which rating earns the most revenue?
 - They want you to prepare a visualization that supports your finding.
 
 - It is then up to you to think of 2 additional hypotheses to test that your stakeholder may want to know.

Some example hypotheses you could test:

 - Do movies that are over 2.5 hours long earn more revenue than movies that are 1.5 hours long (or less)?
 
 - Do movies released in 2020 earn less revenue than movies released in 2018?

  - How do the years compare for movie ratings?

 - Do some movie genres earn more revenue than others?

 - Are some genres higher rated than others?
 - etc.

## Specifications

#### Your Data

- A critical first step for this assignment will be to retrieve additional movie data to add to your SQL database.
  - You will want to use the TMDB API again and extract data for additional years.
  - You may want to review the optional lesson from Week 1 on "Using Glob to Load Many Files" to load and combine all of your API results for each year.
- However, trying to extract the TMDB data for all movies from 2000-2022 could take >24 hours!
- To address this issue, you should EITHER:
   - Define a smaller (but logical) period of time to use for your analyses (e.g., last 10 years, 2010-2019 (pre-pandemic, etc).
   - OR coordinate with cohort-mates and divide the API calls so that you can all download the data for a smaller number of years and then share your downloaded JSON data.

## Deliverables

- You should use the same project repository you have been using for Parts 1-3 (for your portfolio).
  - Create a new notebook in your project repository just for the hypothesis testing (like "Part 4 - Hypothesis Testing.ipynb")
  - Make sure the results and visualization for all 3 hypotheses are in your notebook.

Please submit the link to your GitHub repository for this assignment.

---